In [0]:
import matplotlib.pyplot as plt

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.applications import VGG16

In [0]:
# Predict individual image
def predictOneImage(x_test, y_test, ans, p_index):
  first_image = x_test[p_index]
  plt.imshow(first_image)
  plt.show()

  count = 0
  for a in ans[p_index]:
      print(labels[count], "=", round(a*100, 2))
      count = count + 1
      
  print("------------------")
  print("True Label:", labels[y_test[p_index][0]])

In [0]:

# https://www.cs.toronto.edu/~kriz/cifar.html
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

labels = {
    0: "plane",
    1: "car  ",
    2: "bird ",
    3: "cat  ",
    4: "deer ",
    5: "dog  ",
    6: "frog ",
    7: "horse",
    8: "ship ",
    9: "truck",    
}

train_size, test_size = x_train.shape[0], x_test.shape[0]
height, width, depth = x_train.shape[1], x_train.shape[2], x_train.shape[3]

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

In [0]:
# Visual the data

index = 0

print(x_train[index])
first_image = x_train[index]
plt.imshow(first_image)
plt.show()

print("Label:", labels[y_train[index][0]])

In [0]:
# For Dense Neural Network
# reshape the 32x32x3 pixels to a 3072 array
# and normalize the number by 255

dim = height * width * depth
x_trainp = x_train.reshape(train_size, dim)
x_testp = x_test.reshape(test_size, dim)

normalize = 255
x_trainp = x_trainp.astype('float32') / normalize
x_testp = x_testp.astype('float32') / normalize

print(x_trainp.shape[0], 'train samples')
print(x_testp.shape[0], 'test samples')

In [0]:
# convert class vectors to binary class matrices: one-hot encoding
num_classes = 10

y_trainp = keras.utils.to_categorical(y_train, num_classes)
y_testp = keras.utils.to_categorical(y_test, num_classes)

print("Encoded Label:", y_trainp[index])

In [0]:
# Dense NN with multiple hidden layers
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(dim,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.summary()

In [0]:
batch_size = 128
epochs = 10

# Set aside the first 10,000 samples as the validation set
val_size = 10000
x_valp = x_trainp[:val_size]
partial_x_trainp = x_trainp[val_size:]

y_valp = y_trainp[:val_size]
partial_y_trainp = y_trainp[val_size:]


history = model.fit(partial_x_trainp, partial_y_trainp,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valp, y_valp))

In [0]:
# Do the prediction on test set
score = model.evaluate(x_testp, y_testp, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ans = model.predict(x_testp)

In [0]:
# Predict individual image
p_index = 666
predictOneImage(x_test, y_test, ans, p_index)

In [0]:
# Use CNN and reshape the 32x32x3 pixels 
# and normalize the number by 255

x_trainCnn = x_train.reshape(train_size, height, width, depth)
x_testCnn = x_test.reshape(test_size, height, width, depth)

x_trainCnn = x_trainCnn.astype('float32') / normalize
x_testCnn = x_testCnn.astype('float32') / normalize

print(x_trainCnn.shape[0], 'train samples')
print(x_testCnn.shape[0], 'test samples')

In [0]:
# Use CNN
cnn = Sequential()

cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))

cnn.add(Flatten())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(10, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

cnn.summary()

In [0]:
# Set aside the first 10,000 samples as the validation set
x_valCnn = x_trainCnn[:val_size]
partial_x_trainCnn = x_trainCnn[val_size:]


history = cnn.fit(partial_x_trainCnn, partial_y_trainp,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valCnn, y_valp))

In [0]:
# Do the prediction on test set
score = cnn.evaluate(x_testCnn, y_testp, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

cnnAns = cnn.predict(x_testCnn)

In [0]:
# Predict individual image
predictOneImage(x_test, y_test, cnnAns, p_index)

In [0]:
# Use pre-trained VGG Imagenet CNN layers

# include_top=False: doesn't include the densely connected classifier layers on top of the network
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(height, width, depth))
conv_base.trainable = True

# Fine-tune the last 3 convolutional layers (i.e. block5)
# Usually fine-tune the last few convolutional layers as it's more higher level (abstract) features to the problem
# while the first few layers are more generic and re-usable features
for layer in conv_base.layers:
  if layer.name[:6] != 'block5':
    layer.trainable = False
    
conv_base.summary()    

In [0]:
# Use CNN
pModel = Sequential()

pModel.add(conv_base)
pModel.add(Flatten())
pModel.add(Dense(64, activation='relu'))
pModel.add(Dense(10, activation='softmax'))

pModel.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

pModel.summary()

In [0]:
history = pModel.fit(partial_x_trainCnn, partial_y_trainp,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_valCnn, y_valp))

In [0]:
# Do the prediction on test set
score = pModel.evaluate(x_testCnn, y_testp, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

pAns = pModel.predict(x_testCnn)

In [0]:
# Predict individual image
predictOneImage(x_test, y_test, pAns, p_index)